In [1]:
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import os
from IPython.display import display, HTML, display_html #usefull to display wide tables
from pyspark_dist_explore import Histogram, hist, distplot, pandas_histogram
from pyspark.sql import functions as F, types, SQLContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
from pyspark.sql import SparkSession
app_name = "final_project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [11]:
sqlContext = SQLContext(sc)

In [5]:
# TODO: Replace <FILL IN> with appropriate code
def parse_point(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    values = point.split(',')[1:]
    #values = filter(None, values)
    indices = range(len(values))
    return zip(indices,values)

print(list(parse_point(toy_rawDF.select('text').first()[0])))

NameError: name 'toy_rawDF' is not defined

In [6]:
from pyspark.sql.functions import udf, split
from pyspark.sql.types import ArrayType, StructType, StructField, LongType, StringType, FloatType, DoubleType


parse_point_udf = udf(parse_point, ArrayType(StructType([StructField('_1', LongType()),StructField('_2', StringType())])))

def parse_raw_df(raw_df):
    """Convert a DataFrame consisting of rows of comma separated text into labels and feature.


    Args:
        raw_df (DataFrame with a 'text' column): DataFrame containing the raw comma separated data.

    Returns:
        DataFrame: A DataFrame with 'label' and 'feature' columns.   
  
    """
    return (raw_df.select(split(raw_df.text,',').getItem(0).cast("double").alias('label'),
                         parse_point_udf(raw_df.text).alias('features'))
                        .cache())

In [7]:
def create_one_hot_dict(input_df):
    """Creates a one-hot-encoder dictionary based on the input data.

    Args:
        input_df (DataFrame with 'features' column): A DataFrame where each row contains a list of
            (featureID, value) tuples.

    Returns:
        dict: A dictionary where the keys are (featureID, value) tuples and map to values that are
            unique integers.
    """
    input_distinct_feats_df = input_df.select(explode(input_df.features)).distinct()
    input_ohe_dict = (input_distinct_feats_df
                     .rdd
                     .map(lambda r: tuple(r[0]))
                     .zipWithIndex().collectAsMap())
    return input_ohe_dict

In [8]:
from pyspark.sql.functions import udf
from pyspark.mllib.linalg import VectorUDT

def ohe_udf_generator(ohe_dict_broadcast):
    """Generate a UDF that is setup to one-hot-encode rows with the given dictionary.

    Note:
        We'll reuse this function to generate a UDF that can one-hot-encode rows based on a
        one-hot-encoding dictionary built from the training data.  Also, you should calculate
        the number of features before calling the one_hot_encoding function.

    Args:
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.

    Returns:
        UserDefinedFunction: A UDF can be used in `DataFrame` `select` statement to call a
            function on each row in a given column.  This UDF should call the one_hot_encoding
            function with the appropriate parameters.
    """
    length = len(ohe_dict_broadcast.value)
    return udf(lambda x: one_hot_encoding(x, ohe_dict_broadcast, length), VectorUDT())


In [9]:
from pyspark.mllib.linalg import SparseVector
def one_hot_encoding(raw_feats, ohe_dict_broadcast, num_ohe_feats):
    """Produce a one-hot-encoding from a list of features and an OHE dictionary.

    Note:
        You should ensure that the indices used to create a SparseVector are sorted.

    Args:
        raw_feats (list of (int, str)): The features corresponding to a single observation.  Each
            feature consists of a tuple of featureID and the feature's value. (e.g. sample_one)
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.
        num_ohe_feats (int): The total number of unique OHE features (combinations of featureID and
            value).

    Returns:
        SparseVector: A SparseVector of length num_ohe_feats with indices equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """
    indices = sorted([ohe_dict_broadcast.value[feat] for feat in raw_feats])
    values = np.ones(len(raw_feats))
    return SparseVector(num_ohe_feats,indices,values)

In [12]:
toy_rawDF = sqlContext.read.text('Toy_Example_Data.csv').withColumnRenamed("value", "text")

In [13]:
toy_rawDF.show()

+--------------------+
|                text|
+--------------------+
|1,4,6.37,2.85,0,A...|
|1,5,7.84,3.91,1,S...|
|1,2,5.5,2.82,1,St...|
|1,3,8.43,1.92,1,E...|
|0,4,6.29,3.43,1,F...|
|1,4,8.36,1.91,1,J...|
|1,4,6.56,3.61,1,B...|
|0,2,7.98,1.96,0,F...|
|1,4,5.52,2.85,0,C...|
|1,3,9.15,3.34,0,F...|
|1,4,6.14,1.82,1,M...|
|0,4,6.87,1.75,1,J...|
|1,3,6.9,3.84,1,Sp...|
|1,2,6.53,1.57,0,A...|
|1,4,9.98,3.66,1,S...|
|0,2,9.78,2.81,1,M...|
|0,2,5.1,3.32,1,St...|
|1,3,5.32,3.62,1,A...|
|1,2,7.16,3.72,1,C...|
|0,3,6.07,2.29,0,R...|
+--------------------+
only showing top 20 rows



In [14]:
weights = [.8, .1, .1]
seed = 42
# Use randomSplit with weights and seed
raw_train_df, raw_validation_df, raw_test_df = toy_rawDF.randomSplit(weights, seed)

# Cache and count the DataFrames
n_train = raw_train_df.cache().count()
n_val = raw_validation_df.cache().count()
n_test = raw_test_df.cache().count()
print(n_train, n_val, n_test, str(n_train + n_val + n_test))

806 94 100 1000


In [15]:
parsed_train_df = parse_raw_df(raw_train_df)
print(parsed_train_df.head())

Row(label=0.0, features=[Row(_1=0, _2='2'), Row(_1=1, _2='5.01'), Row(_1=2, _2='2.41'), Row(_1=3, _2='1'), Row(_1=4, _2='Math'), Row(_1=5, _2='German')])


In [16]:
from pyspark.sql.functions import (explode, col)
num_categories = (parsed_train_df
                    .select(explode('features').alias('features'))
                    .distinct()
                    .select(col('features').getField('_1').alias('featureNumber'))
                    .groupBy('featureNumber')
                    .sum()
                    .orderBy('featureNumber')
                    .collect())

In [17]:
print(num_categories)

[Row(featureNumber=0, sum(featureNumber)=0), Row(featureNumber=1, sum(featureNumber)=402), Row(featureNumber=2, sum(featureNumber)=480), Row(featureNumber=3, sum(featureNumber)=6), Row(featureNumber=4, sum(featureNumber)=68), Row(featureNumber=5, sum(featureNumber)=85)]


In [18]:
ctr_ohe_dict = create_one_hot_dict(parsed_train_df)
num_ctr_ohe_feats = len(ctr_ohe_dict)
print(num_ctr_ohe_feats)
ctr_ohe_dict

682


{(2, '1.55'): 0,
 (1, '7.37'): 1,
 (1, '5.54'): 2,
 (2, '3.79'): 3,
 (1, '9.66'): 4,
 (2, '3.62'): 5,
 (1, '7.38'): 6,
 (1, '9.29'): 7,
 (2, '2.04'): 8,
 (1, '6'): 9,
 (2, '3.45'): 10,
 (2, '3.22'): 11,
 (2, '3.54'): 12,
 (1, '6.46'): 13,
 (1, '7.25'): 14,
 (1, '7.1'): 15,
 (1, '9.07'): 16,
 (2, '3.93'): 17,
 (1, '6.06'): 18,
 (1, '8.48'): 19,
 (2, '1.96'): 20,
 (2, '1.88'): 21,
 (1, '6.98'): 22,
 (5, 'Fine Arts'): 23,
 (1, '6.82'): 24,
 (1, '5.56'): 25,
 (1, '6.6'): 26,
 (1, '9.37'): 27,
 (2, '3.86'): 28,
 (2, '1.98'): 29,
 (1, '5.97'): 30,
 (1, '5.73'): 31,
 (4, 'Electrical Engineering'): 32,
 (1, '7.16'): 33,
 (1, '7.44'): 34,
 (1, '6.43'): 35,
 (2, '1.93'): 36,
 (2, '3.13'): 37,
 (1, '8.26'): 38,
 (1, '9'): 39,
 (1, '6.77'): 40,
 (1, '9.08'): 41,
 (2, '2.72'): 42,
 (1, '7.24'): 43,
 (1, '9.46'): 44,
 (1, '9.63'): 45,
 (2, '2.66'): 46,
 (1, '9.61'): 47,
 (2, '2.52'): 48,
 (1, '5.96'): 49,
 (1, '7.05'): 50,
 (1, '8.45'): 51,
 (1, '7.92'): 52,
 (1, '8.32'): 53,
 (1, '9.43'): 54,
 (2, 

In [19]:
ohe_dict_broadcast = sc.broadcast(ctr_ohe_dict)
ohe_dict_udf =  ohe_udf_generator(ohe_dict_broadcast)
ohe_train_df =  parsed_train_df.select(parsed_train_df.label, ohe_dict_udf(parsed_train_df.features).alias('features'))
#ohe_train_df.show(1)                  

print(ohe_train_df.count())
print(ohe_train_df.show())
print(ohe_train_df.take(1))

806
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(682,[82,113,117,...|
|  0.0|(682,[23,82,185,3...|
|  0.0|(682,[70,82,234,2...|
|  0.0|(682,[82,355,418,...|
|  0.0|(682,[82,462,576,...|
|  0.0|(682,[176,378,418...|
|  0.0|(682,[260,378,418...|
|  0.0|(682,[378,414,447...|
|  0.0|(682,[217,378,522...|
|  0.0|(682,[235,378,536...|
|  0.0|(682,[82,151,152,...|
|  0.0|(682,[82,230,272,...|
|  0.0|(682,[160,227,378...|
|  0.0|(682,[82,153,160,...|
|  0.0|(682,[131,182,348...|
|  0.0|(682,[67,82,137,5...|
|  0.0|(682,[32,137,250,...|
|  0.0|(682,[82,137,154,...|
|  0.0|(682,[117,267,330...|
|  0.0|(682,[82,144,370,...|
+-----+--------------------+
only showing top 20 rows

None
[Row(label=0.0, features=SparseVector(682, {82: 1.0, 113: 1.0, 117: 1.0, 413: 1.0, 502: 1.0, 674: 1.0}))]


In [21]:
ohe_train_rdd = ohe_train_df \
                     .rdd \
                     .cache()

In [22]:
meanDropOut = ohe_train_rdd.map(lambda x: x[0]).mean()
varDropOut = ohe_train_rdd.map(lambda x: x[0]).variance()
print(f"Mean: {meanDropOut}")
print(f"Variance: {varDropOut}")

Mean: 0.6625310173697274
Variance: 0.2235836683927612


In [23]:
BASELINE = np.append(meanDropOut, np.zeros(ohe_train_df.count()))

In [24]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [25]:
def GradientDescent(trainRDD, testRDD, wInit, nSteps = 20, 
                    learningRate = 0.1, verbose = False):
    """
    Perform nSteps iterations of OLS gradient descent and 
    track loss on a test and train set. Return lists of
    test/train loss and the models themselves.
    """
    # initialize lists to track model performance
    train_history, test_history, model_history = [], [], []
    
    # perform n updates & compute test and train loss after each
    model = wInit
    for idx in range(nSteps):  
        ############## YOUR CODE HERE #############
        model = GDUpdate(trainRDD, model, learningRate)
        training_loss = LogLoss(trainRDD, model) 
        test_loss = LogLoss(testRDD, model)
        ############## (END) YOUR CODE #############
        
        # keep track of test/train loss for plotting
        train_history.append(training_loss)
        test_history.append(test_loss)
        model_history.append(model)
        
        # console output if desired
        if verbose:
            print("----------")
            print(f"STEP: {idx+1}")
            print(f"training loss: {training_loss}")
            print(f"test loss: {test_loss}")
            print(f"Model: {[round(w,3) for w in model]}")
    return train_history, test_history, model_history

In [26]:
def plotErrorCurves(trainLoss, testLoss, title = None):
    """
    Helper function for plotting.
    Args: trainLoss (list of MSE) , testLoss (list of MSE)
    """
    fig, ax = plt.subplots(1,1,figsize = (16,8))
    x = list(range(len(trainLoss)))[1:]
    ax.plot(x, trainLoss[1:], 'k--', label='Training Loss')
    ax.plot(x, testLoss[1:], 'r--', label='Test Loss')
    ax.legend(loc='upper right', fontsize='x-large')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Log loss')
    if title:
        plt.title(title)
    plt.show()

In [27]:
def LogLoss(dataRDD, W):
    """
    Compute log loss.
    Args:
        dataRDD - each record is a tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    """
    augmentedData = dataRDD.map(lambda x: (x[0], np.append([1.0], x[1])))
    ################## YOUR CODE HERE ##################
    loss = augmentedData.map(lambda x: (-x[0] * np.log(sigmoid(W.dot(x[1]))) - (1 - x[0]) * np.log(1 - sigmoid(W.dot(x[1])))) ).mean()
    ################## (END) YOUR CODE ##################
    return loss

In [28]:
def GDUpdate(dataRDD, W, learningRate = 0.1):
    """
    Perform one OLS gradient descent step/update.
    Args:
        dataRDD - records are tuples of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    Returns:
        new_model - (array) updated coefficients, bias at index 0
    """
    # add a bias 'feature' of 1 at index 0
    augmentedData = dataRDD.map(lambda x: ( x[0], np.append([1.0], x[1]))).cache()
    
    ################## YOUR CODE HERE ################# 
    grad = augmentedData.map(lambda x: (sigmoid(W.dot(x[1])) - x[0])*x[1]).mean()
    new_model = W - learningRate * grad
    ################## (END) YOUR CODE ################# 
    
    return new_model

In [29]:
nSteps = 5
model = BASELINE
print(f"BASELINE:  Loss = {LogLoss(ohe_train_rdd,model)}")
for idx in range(nSteps):
    print("----------")
    print(f"STEP: {idx+1}")
    model = GDUpdate(ohe_train_rdd, model)
    loss = LogLoss(ohe_train_rdd, model)
    print(f"Loss: {loss}")
    print(f"Model: {[round(w,3) for w in model]}")

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 1024, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/anaconda/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/rdd.py", line 2457, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/anaconda/lib/python3.6/site-packages/pyspark-2.3.1-py3.6.egg/pyspark/rdd.py", line 2457, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/anaconda/lib/python3.6/sit